## Load the relevant collections

In [1]:
import numpy as np
from pymongo import MongoClient
import os
import pickle

In [2]:
# Main
mongo_url = "fill_in"
myclient = MongoClient(mongo_url)
mydb = myclient["TangramsCompGen"]

treatment_col = mydb["treatments"]
factor_types_col = mydb["factor_types"]
factors_col = mydb["factors"]
game_lobbies_col = mydb["game_lobbies"]

## Key util

In [3]:
import string
import random

def random_string():
    selections = "23456789ABCDEFGHJKLMNPQRSTWXYZabcdefghijkmnopqrstuvwxyz"
    new_string = ""
    
    for i in range(17):
        new_string += random.sample(selections, 1)[0]
        
    return new_string

In [4]:
treatments = ["baseline", "full", "no_ji", 'no_ds']

## One-time Use

In [5]:

def add_numeric_factor_values(factor_type_name, value_range):
    # Get the id of the factor type
    factor_type_id = factor_types_col.find({"name" : factor_type_name})[0]["_id"]
    
    # Iterate over each value in the range
    for i in range(value_range[0], value_range[1]):
        str_i = str(i)
        new_factor = {
            "_id" : random_string(),
            "name" : f"game_{i}",
            "value" : str_i,
            "factorTypeId" : factor_type_id
        }
        
        factors_col.insert_one(new_factor)
        
def delete_numeric_factor_values(value_range):
    for i in range(value_range[0], value_range[1]):
        target = {"name" : f"game_{i}"}
        factors_col.delete_one(target)
        
add_numeric_factor_values("gameNum", [500, 700])

In [4]:
def add_botIP_treatment_and_values(treatment_names, dummy_value):
    for treatment in treatment_names:
        # First create the relevant factor type
        new_factor_type = {
            '_id' : random_string(),
            "name" : f'{treatment}_IP',
            'description' : f"IP for treatment {treatment}",
            "required" : False,
            "type" : "String",
            "min" : 0,
            "max" : 1000000
        }
        factor_types_col.insert_one(new_factor_type)
        
        # Next create the relevant factor
        factor_type_id = factor_types_col.find({'name' : f'{treatment}_IP'})[0]["_id"]
        new_factor = {
            '_id' : random_string(),
            "name" : "IP",
            "value" : dummy_value,
            "factorTypeId" : factor_type_id
        }
        factors_col.insert_one(new_factor)
        
treatments = ['old_full']
add_botIP_treatment_and_values(treatments, 'localhost')

## Overwriting Bot IPs to be different

In [7]:
def overwrite_botIP_values(treatment_to_IP):
    for treatment, IP in treatment_to_IP.items():
        factor_type_id = factor_types_col.find({'name' : f'{treatment}_IP'})[0]["_id"]
        factor_filter = {"name" : "IP", 'factorTypeId' : factor_type_id}
        update_values = {
            "$set" : {"value" : IP}
        }
        factors_col.update_one(factor_filter, update_values)

treatment_to_IP = {
    "full" : 'https://e03784040fca.ngrok.app',
    "no_ds" : 'https://e03784040fca.ngrok.app',
    "baseline" : 'https://3e7676ef8571.ngrok.app',
    "no_ji" : 'https://3e7676ef8571.ngrok.app',
    'old_full' : 'https://e03784040fca.ngrok.app',
}

overwrite_botIP_values(treatment_to_IP)

## Creating factor values for new experiment/round

In [6]:
def add_experiment_values(factor_type_name, treatment_names, round_number):
    # Get the id of the factor type
    factor_type_id = factor_types_col.find({"name" : factor_type_name})[0]["_id"]
    
    for treatment_name in treatment_names:
        new_factor = {
            "_id" : random_string(),
            "name" : f"r{round_number}_{treatment_name}",
            "value" : f"r{round_number}_{treatment_name}",
            "factorTypeId" : factor_type_id
        }
        factors_col.insert_one(new_factor)
        
treatments = ['human_model', 'human_human']
add_experiment_values("experimentName", treatments, 4)

In [7]:
def add_annotation_value(factor_type_name, annotation):
    # Get the id of the factor type
    factor_type_id = factor_types_col.find({"name" : factor_type_name})[0]["_id"]
    new_factor = {
        "_id" : random_string(),
        "name" : annotation,
        "value" : annotation,
        "factorTypeId" : factor_type_id
    }
    factors_col.insert_one(new_factor)
        
add_annotation_value("annotation", "may_17")

## Creating treatments and overwriting batches for new round

In [8]:
# Create treatments
ft_to_id = {}
factor_types = ['teamColor', 'playerCount', 'experimentName', 'roundNum', 'numConfigs', 'selectionDuration', 
                'multiRound',  'botTreatment', 'botsCount', 'botIP', 'annotation', 'gameNum', 'full_IP', 'no_ji_IP', 
                'no_ds_IP', 'baseline_IP', 'old_full_IP']
for factor_type in factor_types:
    ft_to_id[factor_type] = factor_types_col.find({"name" : factor_type})[0]["_id"]

ft_to_default = {
    'playerCount' : "duo",
    "teamColor" : "blue",
    "selectionDuration" : "45",
    "multiRound" : "false",
    "roundNum" : "0",
    "numConfigs" : "0",
    'full_IP' : 'IP',
    'no_ji_IP' : 'IP',
    'no_ds_IP' : 'IP',
    'baseline_IP' : 'IP',
    'old_full_IP' : 'IP',
    'botIP' : 'localhost',
    'botTreatment' : 'none',
}

def create_game_treatments(ft_to_default, treatments, round_number, start_value, end_value, annotation):
    for treatment in treatments:
        ft_to_non_default = {
            'experimentName' : f"r{round_number}_{treatment}",
            "annotation" : annotation,
        }
        if treatment == "human_human":
            ft_to_non_default['botsCount'] = "zero"
        else:
            ft_to_non_default['botsCount'] = "one"
            
        for i in range(start_value, end_value):
            factor_ids = []
            for factor_type in factor_types:
                if factor_type in ft_to_default:
                    factor_name = ft_to_default[factor_type]
                elif factor_type in ft_to_non_default:
                    factor_name = ft_to_non_default[factor_type]
                else:
                    factor_name = f"game_{i}"
                    
                factor_ids.append(factors_col.find({
                    "factorTypeId" : ft_to_id[factor_type],
                    "name" : factor_name,
                })[0]["_id"])
                
            new_treatment = {
                "_id" : random_string(),
                "name" : ft_to_non_default["experimentName"] + f"_game_{i}",
                "factorIds" : factor_ids
            }
            
            treatment_col.insert_one(new_treatment)
                
round_idx = 4
annotation = 'may_17'
treatments = ['human_human']
start_value = 0
end_value = 70
create_game_treatments(ft_to_default, treatments, round_idx, start_value, end_value, annotation)

treatments = ['human_model']
start_value = 0
end_value = 700
create_game_treatments(ft_to_default, treatments, round_idx, start_value, end_value, annotation)

In [5]:
# Overwrite treatments in batches
def overwrite_treatments(batch_index, treatments, round_id, start_value):
    batch_id = mydb["batches"].find({"index" : batch_index})[0]["_id"]
    for treatment in treatments:
        dummy_name = f"dummy_{treatment}"
        original_treatment_id = treatment_col.find({"name" : dummy_name})[0]["_id"]
        
        all_matching_lobbies = game_lobbies_col.find(
            {
                "batchId" : batch_id, 
                "treatmentId" : original_treatment_id
            }
        )
        
        for i, curr_lobby in enumerate(all_matching_lobbies):
            new_treatment_name = f"r{round_id}_{treatment}_game_{start_value + i}"
            new_treatment_id = treatment_col.find({"name" : new_treatment_name})[0]["_id"]

            target = {"_id" : curr_lobby["_id"]}
            update_values = {"$set" : {"treatmentId" : new_treatment_id}}
            game_lobbies_col.update_one(target, update_values)
            
def delete_treatments(treatments, round_id, num_games):
    for treatment in treatments:
        for i in range(num_games):
            target = {"name" : f"r{round_id}_{treatment}_game_{i}"}
            treatment_col.delete_one(target)

In [8]:
round_idx = 4

treatments = ['human_human']
start_value = 60
overwrite_treatments(191, treatments, round_idx, start_value)

treatments = ['human_model']
start_value = 600
overwrite_treatments(191, treatments, round_idx, start_value)

In [10]:
treatments = ['human', 'full', 'no_ji', 'no_ps', 'no_ds', 'baseline']
delete_treatments(treatments, 2, 100)

## More targeted overwriting

In [13]:
# Load the games we are interested in overwriting
filepath = os.path.join('post_mortems', 'may_17_incompletes.pkl')
with open(filepath, 'rb') as f:
    new_data = pickle.load(f)
    
filepath = os.path.join('post_mortems', 'may_20_incompletes.pkl')
with open(filepath, 'rb') as f:
    new_data = new_data + pickle.load(f)

treatment_to_games = {}
for config_path in new_data:
    components = config_path[:-5].split('/')
    treatment = components[-2][3:]

    if treatment == "human_human":
        continue

    if treatment not in treatment_to_games:
        treatment_to_games[treatment] = []

    game_num = int(components[-1].split('_')[-1])
    treatment_to_games[treatment].append(game_num)

for treatment, games in treatment_to_games.items():
    print(f"Treatment {treatment}: {len(games)} games remaining")
    for game in games:
        print(treatment, game)

Treatment human_model: 22 games remaining
human_model 229
human_model 174
human_model 156
human_model 205
human_model 209
human_model 170
human_model 128
human_model 101
human_model 249
human_model 252
human_model 122
human_model 233
human_model 226
human_model 148
human_model 206
human_model 127
human_model 223
human_model 293
human_model 291
human_model 266
human_model 319
human_model 303


In [14]:
def overwrite_targeted_treatments(batch_index, treatment_dict, round_id):
    batch_id = mydb["batches"].find({"index" : batch_index})[0]["_id"]
    for treatment, treatment_games in treatment_dict.items():
        dummy_name = f"dummy_{treatment}"
        original_treatment_id = treatment_col.find({"name" : dummy_name})[0]["_id"]
        
        all_matching_lobbies = game_lobbies_col.find(
            {
                "batchId" : batch_id, 
                "treatmentId" : original_treatment_id
            }
        )
        
        for i, curr_lobby in enumerate(all_matching_lobbies):
            if i >= len(treatment_games):
                break

            game_value = treatment_games[i]
            new_treatment_name = f"r{round_id}_{treatment}_game_{game_value}"
            new_treatment_id = treatment_col.find({"name" : new_treatment_name})[0]["_id"]

            target = {"_id" : curr_lobby["_id"]}
            update_values = {"$set" : {"treatmentId" : new_treatment_id}}
            game_lobbies_col.update_one(target, update_values)
            
overwrite_targeted_treatments(189, treatment_to_games, 4)

125

In [12]:
425 / 30

14.166666666666666